# Introduction

Most dynamic equilbrium models in economics do not have closed form solutions.  Instead, numerical methods are used to approximate their behavior. Here I show how Deep-Q Learning with a Natural Advantage Function can be used to approximate the solution to dynamic equilibrium models.

First, I review the neoclassical growth model.  Then I show how to solve the model using the Value Iteration Approach.  Finally, I show how to solve the model using Deep-Q Learning with the Natural Advantage Function.  In the future, I'll compare the accuracy of the Deep-Q Learning to Peturbation Methods and the Value Iteration Approach on more complicated models.

Useful background material is "Comparing Solution Methods for Dynamic Equilibrium Economies" (https://www.econstor.eu/bitstream/10419/100716/1/wp2003-27.pdf) which describes the relative accuracy existing numerical solution methods.  "Continuous Deep Q-Learning with Model-based Acceleration" (https://arxiv.org/pdf/1603.00748.pdf) introduces the Natural Advantage function.

## The Neoclassical Model

The neoclassical growth model was introduced in the 40s to describe long-term growth of economies. It involves a planner for the economy.  The planner faces a bread vs butter problem. She has some capital today.  She needs to decide how much capital society should consume today vs how much capital she should invest so people can consume tomorrow.

The two building blocks are the utility function: how happy society is from consuming $c_t$.  And the production function, how much can society produce given capital $k_t$

Given capital $k_t$ in period $t$, society can produce $z_tk_t^{\alpha}$ capital the next period.  Where $\alpha < 1$, there are diminishing returns. And there is a technology $z_t>0$.  Capital deprecates at rate $\delta$. Therefore the production function looks like

$$z_tk_t^{\alpha} + (1- \delta)k_t$$

The utility function is the $\log$ of consumption, eg there are diminishing returns to consuming.  In more complicated models there would also be leisure and a return on leisure.  Putting these building blocks together the planners problem looks like

$$ \max_{c_t, k_{t+1}} \mathbb{E} \sum_{t=0}^{\inf} (1-\beta)\beta^t \log{c_t}$$

subject to

$$c_t + k_{t+1} = z_tk_t^{\alpha} + (1- \delta)k_t$$ 

The planners problem is to choose $c_t$ and $k_{t+1}$ to maximize utility. 

# Value Iteration Approach

Aruoba and Fernández-Villaverde solve the problem using the Value Iteration Approach here https://github.com/jesusfv/Comparison-Programming-Languages-Economics/blob/master/RBC_Python.py

The difficulty in useing the "Value Iteration Method" is that the state space and decision spaces are continious.  But value itearation is discrete.  The simplest approach reproduced below is to just discretize the space.  That approach is the most accurate.  But also doesn't scale computationally, the complexity increases exponentially with the complexity of the state space. 

The code is reproduced below.  First, we define the dynamics of the economy.

In [3]:
import numpy as np

aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
bbeta  = 0.95        # Discount factor

# Productivity values
vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212],float)

# Transition matrix
mTransition = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                     [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                     [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                     [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                     [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]],float)

Then we reproduce the grids used to solve solve the problem.

In [14]:
capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
outputSteadyState      = capitalSteadyState**aalpha
consumptionSteadyState = outputSteadyState-capitalSteadyState

# We generate the grid of capital
vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.00001)

nGridCapital           = len(vGridCapital)
nGridProductivity      = len(vProductivity)

## 3. Required matrices and vectors

mOutput           = np.zeros((nGridCapital,nGridProductivity),dtype=float)
mValueFunction    = np.zeros((nGridCapital,nGridProductivity),dtype=float)
mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity),dtype=float)
mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity),dtype=float)
expectedValueFunction = np.zeros((nGridCapital,nGridProductivity),dtype=float)

 # 4. We pre-build output for each point in the grid

for nProductivity in range(nGridProductivity):
    mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

print "Output = ", outputSteadyState, " Steady State Capital = ", capitalSteadyState, "  Steady State Consumption = ", consumptionSteadyState
print "len vGridCapital", nGridCapital, "dim mPolicyFunction", mPolicyFunction.shape


Output =  0.562731433871  Steady State Capital =  0.178198287393   Steady State Consumption =  0.384533146479
len vGridCapital 17820 dim mPolicyFunction (17820, 5)


In [19]:
import math
import time

t1=time.time()

maxDifference = 10.0
tolerance = 0.0000001
iteration = 0

log = math.log
zeros = np.zeros
dot = np.dot

while(maxDifference > tolerance):
    expectedValueFunction = dot(mValueFunction,mTransition.T)

    for nProductivity in xrange(nGridProductivity):

        # We start from previous choice (monotonicity of policy function)
        gridCapitalNextPeriod = 0

        for nCapital in xrange(nGridCapital):

            valueHighSoFar = -100000.0
            capitalChoice  = vGridCapital[0]

            for nCapitalNextPeriod in xrange(gridCapitalNextPeriod,nGridCapital):
                consumption = mOutput[nCapital,nProductivity] - vGridCapital[nCapitalNextPeriod]
                valueProvisional = (1-bbeta)*log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity];

                if valueProvisional>valueHighSoFar:
                    valueHighSoFar = valueProvisional
                    capitalChoice = vGridCapital[nCapitalNextPeriod]
                    gridCapitalNextPeriod = nCapitalNextPeriod
                else:
                    break # We break when we have achieved the max


            mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
            mPolicyFunction[nCapital,nProductivity]   = capitalChoice

    maxDifference = (abs(mValueFunctionNew-mValueFunction)).max()

    mValueFunction    = mValueFunctionNew
    mValueFunctionNew = zeros((nGridCapital,nGridProductivity),dtype=float)

    iteration += 1
    if(iteration%10 == 0 or iteration == 1):
        print " Iteration = ", iteration, ", Sup Diff = ", maxDifference
        
print " Iteration = ", iter, ", Sup Duff = ", maxDifference
print " "
print " My Check = ", mPolicyFunction[1000-1,3-1]
print " "

t2 = time.time()

print "Elapse time = is ", t2-t1

 Iteration =  1 , Sup Diff =  9.23064875646e-08
 Iteration =  <built-in function iter> , Sup Duff =  9.23064875646e-08
 
 My Check =  0.146549143696
 
Elapse time = is  0.311843156815


# NAF Approach

Instead of discritizing the policy function we represent the utility of state $S$ (in this case capital $k_t$ and technology $z_t$) using a neural network $V(k_t, z_t)$.  The planners problem is then to select $c_t, k_{t+1}$ to maximize 

$$ \log{c_t} + \beta  \mathbb{E} V_{k_{t+1}, z_{t+1}}$$

However, if $V(k, z)$ is arbritary there will not be an analytical way to solve for (note is this true??) $c_t$.  Therefore, we limit the functional form of $V$ in the following way

$$ Q(z_t, k_t, c_t) = A(z_t, k_t, c_t) +_ V(z_t, k_t)$$
$$ A(z_t, k_t, c_t) = (c_t - \mu(z_t, k_t))' P(k_t) (c_t - \mu(z_t, k_t))$$

$P$ is a state-dependent positive-definite square matrix. This ensures that the maximal value of $A$ is at most 0, so the optimal policy function is to always make select $c_t = \mu(z_t, k_t)$.

We can now apply value iteration on arbritary points